# A Time Series Modeling Guide for Waridi Investments: Using Zillow Data to Identify the Most Profitable Zip Codes for Real Estate Investments


## Group 8 Project Colaborators

**STUDENT NAMES:**  
+ SAMMY SIFUNA  
+ JULIUS CHARLES  
+ WARUCH KURIA  
+ RAEL NDONYE  
+ ALLAN OMONDI  
+ JANET KHAINZA
----------------------------------------------

## Business Understanding
       
### Background
Between `1996` to `2018`, the U.S. real estate market underwent significant transformations. It commenced with an increase in home prices driven by a growing perception of real estate as a valuable long-term investment. However, the Great Recession between `2007` to `2009` brought about extensive declines in home values, triggering foreclosures and eroding trust.
Over time,low interest rates and government interventions helped restore stability and confidence in the market. Urbanization gained momentum as more people gravitated towards cities, younger individuals entered the housing market, and technology assumed a pivotal role. Although certain cities thrived, others faced challenges. Regulatory reforms were introduced to prevent potential crises. Hence, the era marked a period of adaptation and evolution in the dynamics of buying and selling homes.
![Depiction of a Changing Market Prices](images/time-series-analysis.jpg)


### Problem Statement
Waridi Investments, a recently established real estate investment firm, has engaged our services to identify the top `5 zip codes` with the potential for the highest return on investment when they `sell in 5 years`. Their strategic approach is to initiate short-term investments in one of the most thriving real estate markets in the United States of America. The company places significant emphasis on securing sound investments that ensure consistent cash flow, ultimately enabling them to reinvest effectively when the opportune moment arises.

### Objectives
* To determine the top `5 zip codes` that show the highest potential `return on investment (ROI) in 5 years.`
* To determine the `5 least advisable cities` to invest in.
* To identify which `top 5 metropolitan areas` and states have the highest market performance in terms of price growth.
* To determine the best month to invest to maximize ROI.
* To show how factors like economic stability(recession) impact housing market performance.

### Hypotheis 
**Null Hypothesis (H0):** The variables do not help predict future real estate prices

**Alternative Hypothesis (H1):** The variables predict future real estate prices

-------------------------------------

## Data Understanding

### A Brief Description of the Dataset

This data represents median monthly housing sales prices for `265 zip codes in the USA`, over the period of `April 1996 through April 2018` as reported by Zillow.

Each row represents a unique zip code. Each record contains location information and median housing sales prices for each month.

There are `11568 rows and 272 variables`:

+ **RegionID:** Unique index
+ **RegionName:** Unique Zip Code
+ **City:** City in which the zip code is located
+ **State:** State in which the zip code is located
+ **Metro:** Metropolitan Area in which the zip code is located
+ **CountyName:** County in which the zip code is located
+ **SizeRank:** Numerical rank of size of zip code, ranked 1 through 14723
+ **1996-04 through 2018-04:** refers to the median housing sales values for `April 1996` through `April 2018`, that is `265 data points` of monthly data for each zip code

We shall check for the accuracy of our forecasts using `MSE (Mean Squared Error)`. This will provide us with the average error of our forecasts.

### Assumptions:
1. The zip codes are representative of the broader real estate market in the United States.
2. The real estate market conditions during this period were influenced by various economic and socio-political factors.
3. The data covers a combination of urban, suburban, and rural area.
4. The data doesn't reflect significant improvements or renovations made to properties over time that could increase or decrease prices.

### Reviewing and Inspecting the Data

In this step, we load, review, and examine the dataset further

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from scipy import stats
from random import gauss as gs
import math
import datetime
import warnings
matplotlib.rcParams['timezone'] = 'UTC'

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pandas.plotting import autocorrelation_plot, lag_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
import itertools
import warnings
warnings.filterwarnings('ignore')
from matplotlib.pylab import rcParams
%matplotlib inline
plt.style.use('ggplot')

C:\Users\charles4data\anaconda3\envs\learn-env\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


ModuleNotFoundError: No module named 'statsmodels.api'

In [1]:
pip install pmdarima

  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
Note: you may need to restart the kernel to use updated packages.


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.3.1 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.24.4 which is incompatible.


In [ ]:
import pmdarima as pm

In [ ]:
#loading dataset and viewing first 5 rows
data = pd.read_csv('data/zillow_data.csv')

data.head()

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
# Examine the shape of the dataset
data.shape

In [ ]:
# Viewing number of zipcodes in dataset and nan values
print('Total number of zipcodes:', len(data))
data.iloc[:, :20].isna().sum() # up to 20th column

## **Exploratory Data Analysis(EDA) and Feature Engineering**

First, some basic EDA and Feature Engineering will be performed. 
- **N/B:** There will be a lot of EDA and feature engineering required throughout this project, therefore, the processes will be performed as needed.

### **Checking for Duplicates**
- The data has no duplicates.

In [ ]:
# check for duplicates
data.duplicated().sum()

### **Changing `RegionName` to `ZipCode`**

The data types are okay as expected.

In [ ]:
#Inspecting dataframe dtypes
data.iloc[:, :20].dtypes

However, `RegionName` column will be renamed to `ZipCode` for easier recognition.
- Further, the column will be converted to a string.

In [ ]:
# change 'RegionName' column to 'ZipCode' to avoid confusion
data.rename(columns={'RegionName': 'ZipCode'}, inplace=True)

data.ZipCode = data.ZipCode.astype('string')

#confirming the change
print(data.dtypes["ZipCode"])

### **Missing Values**

There are some missing values in the `Metro` and `dates` columns. To deal with this:
- Missing values in the **Metro** column will be replaced with **missing**.  
- Missing values in the dates columns will be backfilled.

In [ ]:
data.isnull().sum()

In [ ]:
# Define a function to explore missing data
def missing(data):
    missing_data = data.isna().sum()
    missing_data = missing_data[missing_data>0]
    return missing_data.to_frame()

# Apply missing_data function to the dataframe
missing(data)

In [ ]:
# replacing the missing values bin Metro with 'missing'
data.Metro.fillna('missing', inplace=True)

# interpolate missing values on date columns
data.interpolate(method="pad", inplace=True)

data.isna().sum().sum()

### **Creating New Columns for Analysis**

#### **Select Top 10 States to Invest in Real Estate**
- The data will first be filtered to choose the ten best states to invest in real estate in the USA. The guiding information has been extracted from this article by [Fit Small Business](https://fitsmallbusiness.com/best-and-worst-states-to-invest-in-real-estate/).
- The top 10 states most suitable for investment that yields high returns in real estate are: 
1. Georgia (GA)
2. Utah (UT)
3. Texas (TX)
4. North Carolina (NC)
5. New Jersey (NJ)
6. Tennessee (TN)
7. Washington (WA)
8. Delaware (DE)
9. Nebraska (NE)
10. Florida (FL).
- This narrows down the number of Zip Codes to **4039**.

In [ ]:
# List of states to invest in
states_to_invest = ['GA', 'UT', 'TX', 'NC', 'NJ', 'TN', 'WA', 'DE', 'NE', 'FL']

# Filter data for the selected states
# new df - data_states created
data_states = data[data['State'].isin(states_to_invest)].copy()

# Drop unnecessary columns
columns_to_drop = ['RegionID', 'SizeRank']
data_states.drop(columns_to_drop, axis=1, inplace=True)

# Reset the index
data_states.reset_index(drop=True, inplace=True)

print('Total Zipcodes in DataFrame:', len(data_states))
data_states.head()

In [ ]:
data_states.shape

#### **Calculating ROI and CV**
- Next, 5 columns will be created:
1. **`CV`**: This will store the coefficient of variation values. The information is used by investors to gauge the degree of volatility or risk in relation to the anticipated returns from their investments.
    - It is calculated as standard deviation divided by the mean.
    - A low CV means there is a lower investment risk, and vice versa for a high CV.
    - 22-year and 5-year CV will be calculated.
2. **`ROI`**: This will store values that show the measure of the profitability of an investment relative to its cost and is typically expressed as a percentage.
    - A high ROI indicates that the gains or profits obtained from the investment are significantly greater than the initial investment itself. 
    - 22-year, 5-year and 3-year ROI will be calculated.

In [ ]:
# 22-year ROI
data_states['ROI_all'] = round(((data_states['2018-04'] / data_states['1996-04']) - 1) * 100, 2)

## 5-year ROI
data_states['ROI_5yr'] = round(((data_states['2018-04'] - data_states['2013-04']) / data_states['2013-01']) * 100, 2)

## 3-year ROI
data_states['ROI_3yr'] = round(((data_states['2018-04'] - data_states['2015-04']) / data_states['2015-01']) * 100, 2)

# standard deviation 22 years
data_states['std_all'] = round(data_states.loc[:,'1996-04':'2018-04'].std(skipna=True, axis=1), 2)

# standard deviation 5 years
data_states['std_5yr'] = round(data_states.loc[:,'2013-04':'2018-04'].std(skipna=True, axis=1), 2)

# mean 22 years
data_states['mean_all'] = round(data_states.loc[:,'1996-04':'2018-04'].mean(skipna=True, axis=1), 2)

# mean 5 years
data_states['mean_5yr'] = round(data_states.loc[:,'2013-04':'2018-04'].mean(skipna=True, axis=1), 2)

# 22-year CV
data_states['CV_all'] = round(data_states['std_all']/data_states['mean_all'] * 100, 2)

# 5-year CV
data_states['CV_5yr'] = round(data_states['std_5yr']/data_states['mean_5yr'] * 100, 2)

# Displaying the columns
data_states[['State', 'ZipCode','ROI_all','ROI_5yr', 'ROI_3yr', 'CV_all', 'CV_5yr']].head()

In [ ]:
#Check for zeros
data_states.describe()

In [ ]:
# viewing the selected States by highest 22-yr ROI
states_roi = round(data_states.groupby('State', group_keys=False).sum()[['ROI_all', 'ROI_5yr', 'ROI_3yr']] / 100, 2)
states_roi.sort_values(by=['ROI_all'], ascending=False)

**Observations**  
Even though the analysis will be focussed on Zip Codes, the states' ROI analysis provides a general overview of zip codes to expect in the top-performing list.
- The State of Florida has the highest ROI for all specified periods.
- Even though New Jersey comes third in 22-year ROI, it performs poorly in the 5-year and 3-year ROI.
- Delaware State has the lowest ROI values for all periods.


In [ ]:
# Create a bar chart of the top 10 zip codes by ROI, ROI_5yr, and ROI_3yr
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
states_roi.plot(kind='bar', ax=ax)

# Set the title, labels, and ticks
plt.title("Comparative Analysis of States by ROI_all, ROI_5yr, and ROI_3yr")
ax.set_ylabel("ROI")
ax.set_xlabel("States")
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

plt.savefig('images/ROI_comp.jpg')
# Show the bar chart
plt.show()

In [ ]:
# displaying states by CV
states_cv = round(data_states.groupby('State', group_keys=False).sum()[['CV_all', 'CV_5yr']] / 100, 2)
states_cv.sort_values(by=['CV_all'], ascending=False)

**Observations**
- Again, Florida State has the highest CV values. This is an indication that even though the return on investment is great, it is still a risky state to invest in. Therefore, zip codes in this state will be investigated with a skeptical attitude.
- Delaware has low returns and low risk. 

In [ ]:
# Create a bar chart of the top 10 zip codes by CV_all and CV_5yr
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
states_cv.plot(kind='bar', ax=ax)

# Set the title, labels, and ticks
plt.title("Comparative Analysis of States by CV")
ax.set_ylabel("CV")
ax.set_xlabel("States")
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

plt.savefig('images/CV_comp.jpg')
# Show the bar chart
plt.show()

In [ ]:
data_states.head()

In [ ]:
data_states.nunique()

## Preparation for Modelling
- The next lines of code will rank zip codes by different ROI and CV periods.  
- This will give an overview of how the zip codes performed individually.
- The codes will filter out the top 10 zip codes by the given criteria.

### **1. All ROI Periods**
- Extract top 10 zip codes by `ROI_all` in descending order

In [ ]:
# Extract top 10 zip codes by ROI_all in descending order
top_10_zipcodes_roi_all = data_states.sort_values(by='ROI_all', ascending=False).head(10)

# Plotting the graph
fig, ax = plt.subplots(figsize=(8, 5))
top_10_zipcodes_roi_all.plot(kind='bar', x='ZipCode', y=['ROI_all', 'ROI_5yr', 'ROI_3yr'], ax=ax)
ax.set_xlabel('Zip Code')
ax.set_ylabel('ROI')
ax.set_title('Top 10 Zip Codes by ROI\n(All Periods)')
plt.savefig('images/ROI_all')
plt.show()

**Observations**  
- Zip Code 31561 seems to be performing extremely well in terms of potential for profits.
- The oter zip codes are somewhere at the same performance level.

### **2. All CV Periods**
- Extract top 10 zip codes by `CV_all` in descending order

In [ ]:
# Extract top 10 zip codes by CV_all in descending order
top_10_zipcodes_cv_all = data_states.sort_values(by='CV_all', ascending=False).head(10)

# Plotting the graph
fig, ax = plt.subplots(figsize=(12, 5))
top_10_zipcodes_cv_all.plot(kind='bar', x='ZipCode', y=['CV_all', 'CV_5yr'], ax=ax)
ax.set_xlabel('Zip Code')
ax.set_ylabel('ROI')
ax.set_title('Top 10 Zip Codes by CV\n(All Periods)')
plt.savefig('images/CV_all')
plt.show()

**Observations**
- Zip code 28039 has the highest CV for the past five years.
- 31527 has the highest overall CV.
- The goal is to choose the zip codes with least CV. 31027, 19962 and 99163 may fit into this criteria,

### **3. ROI and CV - 22 years**
- Extract top 10 zip codes by `ROI_all` in descending order.
- Plot both `ROI_all` and `CV_all`.

In [ ]:
# Extract top 10 zip codes by ROI_all in descending order
top_10_zipcodes_roi_cv_all = data_states.sort_values(by='ROI_all', ascending=False).head(10)

# Plotting the graph
fig, ax = plt.subplots(figsize=(12, 5))
top_10_zipcodes_roi_cv_all.plot(kind='bar', x='ZipCode', y=['ROI_all', 'CV_all'], ax=ax)
ax.set_xlabel('Zip Code')
ax.set_ylabel('Value')
ax.set_title('ROI vs CV\n(1996 to 2018)')
plt.savefig('images/ROI_CV_all')
plt.show()

### **4. ROI and CV - 5 years**
- Extract top 10 zip codes by `ROI_5yr` in descending order.
- Plot both `ROI_5yr` and `CV_5yr`.

**Observations**
- This graph is unclear on how the two values compare because the extreme ROI value for 31561 seems to overpower the rest.

In [ ]:
# Extract top 10 zip codes by ROI_all in descending order
top_10_zipcodes_roi_cv_5yr = data_states.sort_values(by='ROI_all', ascending=False).head(10)

# Plotting the graph
fig, ax = plt.subplots(figsize=(12, 5))
top_10_zipcodes_roi_cv_5yr.plot(kind='bar', x='ZipCode', y=['ROI_5yr', 'CV_5yr'], ax=ax)
ax.set_xlabel('Zip Code')
ax.set_ylabel('Value')
ax.set_title('ROI vs CV\n(5-year Period)')
plt.savefig('images/ROI_CV_5yr')
plt.show()

**Observations**
- Here, 31527 stands out again with a high 5-year ROI. The CV, however, is unfortunately also high.
- 7302, 30317 and 78702 have high ROIs but higher than most CV values.
- This graph seems to give a better comparable visualization and will be used to form the time series for modelling.

In [ ]:
top_10_zipcodes_roi_cv_5yr.head()

In [ ]:
top_10_zipcodes_roi_cv_5yr.shape

In [ ]:
# confirming the list of Zip Codes
top_10_zipcodes_roi_cv_5yr["ZipCode"]

### Identifying the Zip Codes by City, County, Metro  and State
|Zip Code| City | County | Metro | State |
| ------ | :---------- | :------ | :----- | :----- |
| 31561  | Sea Island | Glynn County | Brunswick | Georgia |
| 31527  | Jekyll Island | Glynn County | Brunswick | Georgia |
| 7302   | Jersey City | Hudson County | New York |  New Jersey |
| 30139  | Fairmount | Gordon County | Calhoun | Georgia |
| 76234  | Decatur | Wise County | Dallas-Fort Worth |  Texas |
| 19951  | Harbeson | Sussex County | Salisbury |  Delaware |
| 98851  | Soap Lake | Grant County | Moses Lake |  Washington |
| 30317  | Atlanta | DeKalb County | Atlanta | Georgia |
| 78702  | Austin | Travis | Austin | Texas |
| 28762  | Old Fort | McDowell | Marion | North Carolina |

In [ ]:
final_df = top_10_zipcodes_roi_cv_5yr.drop(['City', 'State', 'Metro', 'CountyName',
                                     'ROI_all', 'ROI_3yr',
                                       'std_all', 'std_5yr',
                                       'mean_all','mean_5yr',
                                       'CV_all'], axis=1)

In [ ]:
final_df.head()

In [ ]:
final_df.info()

### Converting to Time Series

In [ ]:
time_df= final_df.melt(id_vars=['ZipCode', 'ROI_5yr', 'CV_5yr'], var_name='Time', value_name='Sale Price')
# time_df['Time'] = pd.to_datetime(final_df['Time'], infer_datetime_format=True)
time_df['Time'] = pd.to_datetime(time_df['Time'])
time_df.set_index('Time', inplace=True)
time_df = time_df.pivot_table('Sale Price', ['Time'], 'ZipCode')
time_df.head()

In [ ]:
time_df.index.name

## Modelling

The line graph below shows the historical average home values for the selected zip codes over the dataset's time period. 

In [ ]:
time_df.plot(figsize=(8,5))
plt.xlabel('Time', fontsize=14)
plt.ylabel('Average Home Value', fontsize=14)
plt.title('Historical Average Home Values for Zipcodes of Interest')
plt.savefig('images/hist_avg')
plt.legend();

**Observations** (changed from 31527 to 7302)
- We shall remove **`31561`** and **`7302`**. They are outliers in the dataset and have dramatic trends over the period under review.

In [ ]:
top_8_df = time_df.drop(['31561','7302'], axis=1)
top_8_df.plot(figsize=(8,5))
plt.xlabel('Year', fontsize=14)
plt.ylabel('Average Home Value', fontsize=14)
plt.title('Historical Average Home Values for Zipcodes of Interest')
plt.savefig('images/hist_avg_top8')
plt.legend();

In [ ]:
# confirming that 8 zipcodes are left
top_8_df.head()

### Adjusting the Review Period (changed month to April)
- The was a market recession during the period 2008 - 2011.
- As a result, the analysis will consider data from April 2012 to April 2018.
- New DataFrame is named **`recent_time**.

In [ ]:
recent_time = top_8_df['2012-03-01':]
recent_time.plot(figsize=(8, 5))
plt.xlabel('Year', fontsize=14)
plt.ylabel('Average Home Value', fontsize=14)
plt.title('Historical Average Home Values for Zipcodes of Interest')
plt.legend()
plt.savefig('images/hist_avg_recent_time');

In [ ]:
recent_time.head()

In [ ]:
# Converting columns to str
recent_time.columns = recent_time.columns.astype(str)
zipcode_list =  top_8_df.columns.astype(str)

recent_time.columns

### 5.1 Seasonality
- The following function checks for and removes seasonality using the `seasonal_decompose` function. 
- Twelve (12) periods are specified to represent a year, the duration of a season in the data.
- After extracting the **trend**, **seasonality** and **residuals**, it plots each of these components in separate subplots. 
- It is important to remove seasonality and trend because If they are part of the time series, there will be effects in the forecast value.

In [ ]:
def seasonal_decomposition(data):
    decomposition = seasonal_decompose(data, period=12)

    # Gather the trend, seasonality, and residuals
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    # Plot gathered statistics
    plt.figure(figsize=(12,8))
    plt.subplot(411)
    plt.plot(data, label='Original', color='blue')
    plt.legend(loc='best')
#     plt.xticks(data.index, data.index.year)
    
    plt.subplot(412)
    plt.plot(trend, label='Trend', color='blue')
    plt.legend(loc='best')
#     plt.xticks(data.index, data.index.year)
    
    plt.subplot(413)
    plt.plot(seasonal,label='Seasonality', color='blue')
    plt.legend(loc='best')
#     plt.xticks(data.index, data.index.year)
    
    plt.subplot(414)
    plt.plot(residual, label='Residuals', color='blue')
    plt.legend(loc='best')
    plt.xticks(data.index, data.index.year)
    
    # Format x-axis labels to show years only
    plt.xticks(data.index, data.index.year)
    
    # Adjust layout for the last subplot to avoid squeezing x-axis labels
    plt.subplots_adjust(bottom=0.8)

    plt.tight_layout()

#### i.) Zipcode 19951 (Harbeson, Delaware)

In [ ]:
seasonal_decomposition(recent_time['19951'])
plt.savefig('images/seas_dec_19951');

#### ii.) Zipcode 28762 (Old Fort, North Carolina)

In [ ]:
seasonal_decomposition(recent_time['28762'])
plt.savefig('images/seas_dec_28762');

#### iii.) Zipcode 30139 (Fairmount, Georgia)

#### iv.) Zipcode 30317 (Atlanta, Georgia)

#### v.) Zipcode 31527 (Brunswick, Georgia)

#### vi.) Zipcode 76234 (Decatur, Texas)

#### vii.) Zipcode 78702 (Austin, Texas)

#### viii.) Zipcode 98851 (Soap Lake, Washington)

The above plots gives a better view of our series componetent, with seasnoality, trend and residual patterns

Find a way to scale up and have time series show

**5.2 Stationarity**

In [ ]:
#
def check_stationarity(data):
    result = adfuller(data)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

In [ ]:
check_stationarity(recent_time['19951'])

It Seems all Data is non-stationary. So let's make it stationary

In [ ]:
def detrend(data):
    '''function returns a stationary series'''

    diff_series = data.diff(1).diff(1).dropna()
    return diff_series
detrended_series = detrend(recent_time['19951'])
detrended_series.head()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(detrended_series)
plt.title('Differenced Data for Zip Code 19951')
plt.xlabel('Time')
plt.ylabel('Differenced Value')
# plt.savefig(images/)
plt.show()

In [ ]:
check_stationarity(detrended_series)

p-value is less than 0.05, hence the series is stationary.

**5.4 Autocorrelation and partial correlation of detrended series**

In [ ]:
plt.figure(figsize=(8, 5))
plt.subplot(211)
plot_acf(detrended_series, ax=plt.gca(), lags=20)
plt.title('Autocorrelation Function (ACF)')

plt.subplot(212)
plot_pacf(detrended_series, ax=plt.gca(), lags=20)
plt.title('Partial Autocorrelation Function (PACF)')

plt.tight_layout()
plt.show()

**5.5 Time Series modeling**

Since our data has seasonality we use SARIMAX model.

In [ ]:
#using auto_arima- it does a random search for the best pdq,PDQS
sarima_model = pm.auto_arima(detrended_series,
                             m=12,
                             seasonal=True,
                             start_p=0,
                             start_q=0,
                             start_P=0,
                             start_Q=0,
                             max_order=6,
                             test='adf',
                             error_action='warn',
                             suppress_warnings=True,
                              stepwise=True,
                              trace=False)

The Akaike Information Criterion (AIC) tests the goodness of fit.It rewards models that achieve a high goodness-of-fit with little complexity

A model that fits the data very well while using lots of features will be assigned a larger AIC score than a model that uses fewer features to achieve the same goodness-of-fit.

In [ ]:
print(sarima_model.summary())

Will look for other values of pdq and s that might lower AIC value.

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 2

p = d = q = range(0,2)

# Generate all different combinations of p, d , q and s
pdq = list(itertools.product(p, d, q))
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            model = sm.tsa.statespace.SARIMAX(detrended_series,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            output = model.fit()
            ans.append([comb, combs, output.aic])

        except:
            continue

In [ ]:
ans_df = pd.DataFrame(ans, columns=['pdq','pdqs', 'aic'])
ans_df

In [ ]:
ans_df.loc[ans_df['aic'].idxmin()]

In [ ]:
# Split your data into training and test sets
train_size = 0.8
split_index = round(len(detrended_series) * train_size)
train = detrended_series[:split_index]
test = detrended_series[split_index:]

# Build and fit your model using the training data
model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
results = model.fit()

print(results.summary())


# Evaluate the model using the test set and calculate performance metrics
##forecast = results.get_forecast(steps=len(test))
##forecasted_values = forecast.predicted_mean

# Calculate the performance metrics and compare forecasts to actual values
##rmse = np.sqrt(mean_squared_error(test, forecasted_values))
##print(f"RMSE: {rmse}")

#plt.figure(figsize=(12, 6))
#plt.plot(test, label='Actual')
#plt.plot(forecasted_values, label='Forecast', color='blue')
#plt.xlabel('Time')
#plt.ylabel('Differenced Value')
#plt.title('Actual vs Forecasted Values')
#plt.legend()
#plt.show()

**6 Model Diagnostics**

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()

**6.1 Forecasting and Model Evaluation**

We compare predicted values to real values of the time series, which will help us understand the accuracy of our forecasts

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2016-01-01'), dynamic=False)

pred_ci = pred.conf_int() # this gives us the confidence interval for our forecasts

In [ ]:
plt.figure(figsize = (15,8))
ax = detrended_series['2013':].plot(label='observed',color='black')
pred.predicted_mean.plot(ax=ax, label='Forecast', color='red',alpha=0.8)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=0.5)

ax.set_title('Dynamic Forecast from 2015-2018')
ax.set_xlabel('Date')
ax.set_ylabel('Amount')
plt.legend()

plt.show()

**6.1.1 Dynamic Focusing**

In [ ]:
pred_dynamic = results.get_prediction(start=pd.to_datetime('2015-01-01'), dynamic=True)
pred_dynamic_ci = pred_dynamic.conf_int()

In [ ]:
plt.figure(figsize = (15,8))
ax = detrended_series['2012':].plot(label='observed',color='black')
pred_dynamic.predicted_mean.plot(ax=ax, label='Forecast', color='red',alpha=0.8)

ax.fill_between(pred_dynamic_ci.index,
                pred_dynamic_ci.iloc[:, 0],
                pred_dynamic_ci.iloc[:, 1], color='k', alpha=.5)

ax.set_title('Non dynamic Forecast from 2015-2018')
ax.set_xlabel('Date')
ax.set_ylabel('Amount')
plt.legend()

plt.show()

**Zipcode 19951**

In [ ]:
# Apply differencing
differenced_data2 = recent_time['19951'].diff(periods=1).dropna()

In [ ]:
# Identifying range for pdq values
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
# Train Test Split
train_size=0.8
split_indx = round(len(differenced_data2) * train_size)
train = differenced_data2[:split_indx]
test = differenced_data2[split_indx:]

# Find lowest AIC for pdq/pdqs combinations
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
        except:
            continue

ans_df2 = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df2.loc[ans_df['aic'].idxmin()]

In [ ]:
model2 = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 0, 12))
results2 = model.fit()

print(results2.summary())

In [ ]:
results2.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Plugging optimal parameters into SARIMA model

SARIMA_MODEL2 = sm.tsa.statespace.SARIMAX(train,
                order=(1,1,1),
                seasonal_order=(1,1,0,12),
                enforce_stationarity=False,
                enforce_invertibility=False)

output = SARIMA_MODEL2.fit()


# Getting test set predictions and confidence intervals from 09/2016 to 04/2018

preds = output.get_prediction(start=pd.to_datetime('2016-09-01'), end=pd.to_datetime('2018-04-01'), dynamic=True)
pred_conf = preds.conf_int()

# Plot observed values
ax = differenced_data2.plot(label='Observed')

# Plot predicted values
preds.predicted_mean.plot(ax=ax, label='Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='lightgreen', alpha=0.5, label='Confidence Interval')

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Sale Prices')
ax.set_title('Model Performance on Zipcode 59718 Test Data')
plt.legend()

plt.show()


#### **Zipcode 28762**

In [ ]:
differenced_data3 = recent_time['28762'].diff(periods=1).dropna()

In [ ]:
p= d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
train_size=0.8
split_indx = round(len(differenced_data3) * train_size)
train = differenced_data3[:split_indx]
test = differenced_data3[split_indx:]

# Find lowest AIC for pdq/pdqs combinations
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
        except:
            continue

ans_df3 = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df3.loc[ans_df['aic'].idxmin()]

In [ ]:
model3 = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 0, 12))
results3 = model.fit()

print(results3.summary())

In [ ]:
results3.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Plugging optimal parameters into SARIMA model

SARIMA_MODEL3 = sm.tsa.statespace.SARIMAX(train,
                order=(1,1,1),
                seasonal_order=(1,1,0,12),
                enforce_stationarity=False,
                enforce_invertibility=False)

output = SARIMA_MODEL3.fit()


# Getting test set predictions and confidence intervals from 09/2016 to 04/2018

preds = output.get_prediction(start=pd.to_datetime('2016-09-01'), end=pd.to_datetime('2018-04-01'), dynamic=True)
pred_conf = preds.conf_int()

# Plot observed values
ax = differenced_data2.plot(label='Observed')

# Plot predicted values
preds.predicted_mean.plot(ax=ax, label='Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='lightgreen', alpha=0.5, label='Confidence Interval')

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Sale Prices')
ax.set_title('Model Performance on Zipcode 59718 Test Data')
plt.legend()

plt.show()

**Zipcode 30139**

In [ ]:
differenced_data4 = recent_time['30139'].diff(periods=1).dropna()

In [ ]:
p= d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
train_size=0.8
split_indx = round(len(differenced_data4) * train_size)
train = differenced_data4[:split_indx]
test = differenced_data4[split_indx:]

# Find lowest AIC for pdq/pdqs combinations
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
        except:
            continue

ans_df4 = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df4.loc[ans_df['aic'].idxmin()]

In [ ]:
import itertools

# Identifying range for pdq values
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

base_zip = recent_time['7302']

# Train Test Split
train_size = 0.8
split_indx = round(len(base_zip) * train_size)
train = base_zip[:split_indx]
test = base_zip[split_indx:]

# Find lowest AIC for pdq/pdqs combinations
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
        except:
            continue

ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
best_model = ans_df.loc[ans_df['aic'].idxmin()]

print("Best model parameters:")
print(best_model)

In [ ]:
model4 = SARIMAX(train, order=(1, 0, 1), seasonal_order=(1, 1, 1, 12))
results4 = model.fit()

print(results3.summary())

In [ ]:
results4.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Plugging optimal parameters into SARIMA model

SARIMA_MODEL4 = sm.tsa.statespace.SARIMAX(train,
                order=(1,1,1),
                seasonal_order=(1,1,0,12),
                enforce_stationarity=False,
                enforce_invertibility=False)

output = SARIMA_MODEL4.fit()


# Getting test set predictions and confidence intervals from 09/2016 to 04/2018

preds = output.get_prediction(start=pd.to_datetime('2016-09-01'), end=pd.to_datetime('2018-04-01'), dynamic=True)
pred_conf = preds.conf_int()

# Plot observed values
ax = differenced_data2.plot(label='Observed')

# Plot predicted values
preds.predicted_mean.plot(ax=ax, label='Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='lightgreen', alpha=0.5, label='Confidence Interval')

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Sale Prices')
ax.set_title('Model Performance on Zipcode 59718 Test Data')
plt.legend()

plt.show()

**Zipcode 30317**

In [ ]:
differenced_data5 = recent_time['30317'].diff(periods=1).dropna()

In [ ]:
p= d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
train_size=0.8
split_indx = round(len(differenced_data4) * train_size)
train = differenced_data5[:split_indx]
test = differenced_data5[split_indx:]

# Find lowest AIC for pdq/pdqs combinations
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
        except:
            continue

ans_df5 = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df5.loc[ans_df['aic'].idxmin()]

In [ ]:
model5 = SARIMAX(train, order=(1, 0, 1), seasonal_order=(1, 1, 1, 12))
results5 = model.fit()

print(results5.summary())

In [ ]:
results5.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Plugging optimal parameters into SARIMA model

SARIMA_MODEL5 = sm.tsa.statespace.SARIMAX(train,
                order=(1,1,1),
                seasonal_order=(1,1,0,12),
                enforce_stationarity=False,
                enforce_invertibility=False)

output5 = SARIMA_MODEL5.fit()

# Getting test set predictions and confidence intervals from 09/2016 to 04/2018

preds = output.get_prediction(start=pd.to_datetime('2016-09-01'), end=pd.to_datetime('2018-04-01'), dynamic=True)
pred_conf = preds.conf_int()

# Plot observed values
ax = differenced_data2.plot(label='Observed')

# Plot predicted values
preds.predicted_mean.plot(ax=ax, label='Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='lightgreen', alpha=0.5, label='Confidence Interval')

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Sale Prices')
ax.set_title('Model Performance on Zipcode 59718 Test Data')
plt.legend()

plt.show()

**Zipcode 76234**

In [ ]:
differenced_data6 = recent_time['30317'].diff(periods=1).dropna()

In [ ]:
# Identifying range for pdq values
p= d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
# Train Test Split
train_size=0.8
split_indx = round(len(differenced_data6) * train_size)
train = differenced_data6[:split_indx]
test = differenced_data6[split_indx:]

# Find lowest AIC for pdq/pdqs combinations
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
        except:
            continue

ans_df6 = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df6.loc[ans_df['aic'].idxmin()]

In [ ]:
model6 = SARIMAX(train, order=(1, 0, 1), seasonal_order=(1, 1, 1, 12))
results6 = model.fit()

print(results6.summary())

In [ ]:
results6.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Plugging optimal parameters into SARIMA model

SARIMA_MODEL6 = sm.tsa.statespace.SARIMAX(train,
                order=(1,0,1),
                seasonal_order=(1,1,1,12),
                enforce_stationarity=False,
                enforce_invertibility=False)

output = SARIMA_MODEL6.fit()


# Getting test set predictions and confidence intervals from 09/2016 to 04/2018

preds = output.get_prediction(start=pd.to_datetime('2016-09-01'), end=pd.to_datetime('2018-04-01'), dynamic=True)
pred_conf = preds.conf_int()

# Plot observed values
ax = differenced_data6.plot(label='Observed')

# Plot predicted values
preds.predicted_mean.plot(ax=ax, label='Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='lightgreen', alpha=0.5, label='Confidence Interval')

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Sale Prices')
ax.set_title('Model Performance on Zipcode 76234 Test Data')
plt.legend()

plt.show()

**Zipcode 78702**

In [ ]:
differenced_data7 = recent_time['78702'].diff(periods=1).dropna()

In [ ]:
# Identifying range for pdq values
p= d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
# Train Test Split
train_size=0.8
split_indx = round(len(differenced_data7) * train_size)
train = differenced_data7[:split_indx]
test = differenced_data7[split_indx:]

# Find lowest AIC for pdq/pdqs combinations
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
        except:
            continue

ans_df7 = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df7.loc[ans_df['aic'].idxmin()]

In [ ]:
model7 = SARIMAX(train, order=(1, 0, 1), seasonal_order=(1, 1, 1, 12))
results7 = model.fit()

print(results7.summary())

In [ ]:
results7.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Plugging optimal parameters into SARIMA model

SARIMA_MODEL7 = sm.tsa.statespace.SARIMAX(train,
                order=(1,0,1),
                seasonal_order=(1,1,1,12),
                enforce_stationarity=False,
                enforce_invertibility=False)

output = SARIMA_MODEL7.fit()


# Getting test set predictions and confidence intervals from 09/2016 to 04/2018

preds = output.get_prediction(start=pd.to_datetime('2016-09-01'), end=pd.to_datetime('2018-04-01'), dynamic=True)
pred_conf = preds.conf_int()

# Plot observed values
ax = differenced_data7.plot(label='Observed')

# Plot predicted values
preds.predicted_mean.plot(ax=ax, label='Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='lightgreen', alpha=0.5, label='Confidence Interval')

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Sale Prices')
ax.set_title('Model Performance on Zipcode 78702 Test Data')
plt.legend()

plt.show()

**Zipcode 98851**

In [ ]:
# Apply differencing
differenced_data8 = recent_time['98851'].diff(periods=1).dropna()

In [ ]:
# Identifying range for pdq values
p= d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
train_size=0.8
split_indx = round(len(differenced_data2) * train_size)
train = differenced_data8[:split_indx]
test = differenced_data8[split_indx:]

# Find lowest AIC for pdq/pdqs combinations
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
        except:
            continue

ans_df8 = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df8.loc[ans_df['aic'].idxmin()]

In [ ]:
model8 = SARIMAX(train, order=(1, 0, 1), seasonal_order=(1, 1, 1, 12))
results8 = model.fit()

print(results8.summary())

In [ ]:
results8.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Plugging optimal parameters into SARIMA model

SARIMA_MODEL8 = sm.tsa.statespace.SARIMAX(train,
                order=(1,0,1),
                seasonal_order=(1,1,1,12),
                enforce_stationarity=False,
                enforce_invertibility=False)

output = SARIMA_MODEL8.fit()


# Getting test set predictions and confidence intervals from 09/2016 to 04/2018

preds = output.get_prediction(start=pd.to_datetime('2016-09-01'), end=pd.to_datetime('2018-04-01'), dynamic=True)
pred_conf = preds.conf_int()

# Plot observed values
ax = differenced_data8.plot(label='Observed')

# Plot predicted values
preds.predicted_mean.plot(ax=ax, label='Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='lightgreen', alpha=0.5, label='Confidence Interval')

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Sale Prices')
ax.set_title('Model Performance on Zipcode 98851 Test Data')
plt.legend()

plt.show()

**8. fbprophet Model -**

**9. CONCLUSION**

**10 Recommendations**